In [1]:
import pandas as pd
import numpy as np
import joblib

from tqdm.notebook import tqdm
tqdm.pandas()

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from sklearn.metrics import classification_report,accuracy_score
from collections import Counter

C:\Users\arbaz\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:702: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
import sys
sys.path

['C:\\Machine Learning\\desktop\\RA',
 'C:\\Users\\arbaz\\AppData\\Local\\Continuum\\anaconda3\\python37.zip',
 'C:\\Users\\arbaz\\AppData\\Local\\Continuum\\anaconda3\\DLLs',
 'C:\\Users\\arbaz\\AppData\\Local\\Continuum\\anaconda3\\lib',
 'C:\\Users\\arbaz\\AppData\\Local\\Continuum\\anaconda3',
 '',
 'C:\\Users\\arbaz\\AppData\\Roaming\\Python\\Python37\\site-packages',
 'C:\\Users\\arbaz\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages',
 'C:\\Users\\arbaz\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\python_levenshtein-0.12.0-py3.7-win-amd64.egg',
 'C:\\Users\\arbaz\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\arbaz\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\arbaz\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\arbaz\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\arbaz\\.ipython']

In [3]:
df_list = joblib.load("../../Unicredit/Training notebook/df_list.joblib")

In [4]:
df_main = df_list[0][["cp_variation","kribs_head"]]
single_value_list = df_main["kribs_head"].value_counts()[df_main["kribs_head"].value_counts()==1].index.tolist()
df_temp = df_main[df_main["kribs_head"].isin(single_value_list)]
df_main = pd.concat([df_main,df_temp])
df_main.reset_index(inplace=True,drop=True)
df_main.columns = ["text","label"]
df_main.shape

(1849, 2)

In [5]:
df_main.tail()

,text,label
1844,liabilities from unused flight documents curre...,84.11 conditioned advances by french state
1845,other provisions non current provisions curren...,257.09 restructuring
1846,the pension provision non current liabilities ...,222.06 monetary compensation
1847,investment properties at fair value ifrs basis...,1.04 commercial real estate and other buildings
1848,trade receivables from activity current asset ...,41.029999999999994 from rental income


In [6]:
# df_main = pd.read_csv("data/ML.csv")
# df_test = pd.read_csv("data/Test.csv")
# df_sub = pd.read_csv("data/submission_lr.csv")
# df_main.shape,df_test.shape,df_sub.shape

In [7]:
# df_main.loc[df_main["text"].str.contains("#NAME?"),"label"] = "Positive"

In [8]:
# df_main.drop_duplicates(inplace=True)
# df_main.reset_index(inplace=True,drop=True)
# df_main.shape

In [9]:
df_main["label"].value_counts()

90.0 other current liabilities                                          87
42.0 other current assets                                               78
14.0 other non current receivables and assets                           77
23.0 patents, trademarks, etc                                           74
82.0 overdraft/short term bank loan - other banks                       59
                                                                        ..
1.04 commercial real estate and other buildings                          2
222.06 monetary compensation                                             2
32.01 real estate and similar land rights with buildings in progress     2
23.11 trademarks                                                         2
14.11 claims to securityship (depots et cautionnement)                   2
Name: label, Length: 108, dtype: int64

In [10]:
lbl_enc = LabelEncoder()
type_one_hot = lbl_enc.fit_transform(
  df_main["label"].to_numpy().reshape(-1, 1)
)

C:\Users\arbaz\AppData\Roaming\Python\Python37\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=98)
for train_index, test_index in sss.split(df_main["text"], type_one_hot):
#     print("TRAIN:", train_index, "TEST:", test_index)
    train, test = df_main["text"][train_index], df_main["text"][test_index]
    y_train, y_test = type_one_hot[train_index], type_one_hot[test_index]

In [12]:
NUM_CLASSES = len(Counter(df_main["label"]))
NUM_CLASSES

108

## RoBERTa using SimpleTransformers - https://medium.com/swlh/simple-transformers-multi-class-text-classification-with-bert-roberta-xlnet-xlm-and-8b585000ce3a

In [13]:
# import wandb
import logging
from simpletransformers.classification import ClassificationModel, ClassificationArgs

#### Creating train/validation dataset in expected format -> (label,text)

In [14]:
df_train = pd.DataFrame()
df_train["text"] = train
df_train["label"] = y_train
df_train.head()

df_val = pd.DataFrame(test,columns=["text"])
df_val["label"] = y_test
df_val.reset_index(inplace=True,drop=True)
df_val.head(3)

,text,label
0,inventories current asset finished products me...,43
1,financial liabilities non current liabilities ...,75
2,payroll tax liabilities and other liabilities ...,98


#### Setting hyperparameters -> https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model

In [15]:
args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 4,
    "weight_decay": 0,
    "learning_rate": 5e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,
    
    "manual_seed": 98,
    'use_early_stopping': True,
    'early_stopping_patience': 2,
    'evaluate_during_training_verbose':True,
    'early_stopping_metric':'mcc',
    'early_stopping_metric_minimize':True,

#     "process_count": cpu_count() - 2 if cpu_count() > 2 else 1,
    "n_gpu": 1,
}

In [17]:
import torch
print(torch.cuda.is_available())

False


#### Model training

In [ ]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=NUM_CLASSES,use_cuda=False,args=args)
model.train_model(df_train, output_dir='roberta_sa_4', eval_df=df_val)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/185 [00:00<?, ?it/s]

#### Predicting and checking validation outputs

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(df_val)

In [ ]:
result,len(wrong_predictions),len(wrong_predictions)/df_val.shape[0]

In [ ]:
print(classification_report(lbl_enc.inverse_transform(np.argmax(model_outputs,axis=1)),lbl_enc.inverse_transform(df_val["label"])))

In [ ]:
# joblib.dump(model,"trained_model/roberta_base_model_v9.pkl")

#### Predicting test dataset

In [ ]:
test_output = model.predict(df_test["text"])

In [ ]:
submission = pd.DataFrame(lbl_enc.inverse_transform(test_output[0]))
submission.reset_index(inplace=True)
submission.columns = ["id","label"]
submission.head()

## Submission

#### pick last best prediction output and compare differences with current output

In [ ]:
submission_old = pd.read_csv("submission_v8.csv")
submission_old.head(2)

In [ ]:
print(classification_report(submission["label"],submission_old["label"]))

In [ ]:
submission_old["label"].value_counts(normalize=True)

In [ ]:
submission["label"].value_counts(normalize=True)

In [ ]:
df_main["label"].value_counts(normalize=True)

In [ ]:
df_test["new_pred"] = submission["label"]
df_test["old_pred"] = submission_old["label"]

In [ ]:
df_test.head(2)

In [ ]:
df_test[df_test["r_large"]!=df_test["r_base"]].shape

In [ ]:
df_test[df_test["r_large"]!=df_test["r_base"]].head()

In [ ]:
df_test[df_test["new_pred"]!=df_test["old_pred"]].to_excel("check_data.xlsx")

## Save to CSV file

In [ ]:
submission.to_csv('submission_v9.csv', index=False)